In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# 데이터 로드

In [4]:
import os
import pandas as pd

train_data_path ="data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [5]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,981,981
기타 괴롭힘 대화,1094,1094
직장 내 괴롭힘 대화,979,979
협박 대화,896,896


In [6]:
test_data = pd.read_json('data/test.json', orient='index')

test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, t_000 to t_499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


# 데이터 전처리

In [8]:
train_data[train_data.iloc[392]['conversation'] == train_data['conversation']]

,idx,class,conversation
29,29,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...
392,392,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...


In [9]:
train_data.duplicated('conversation').sum()

104

In [10]:
train_data = train_data.drop_duplicates(subset='conversation')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3846 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3846 non-null   int64 
 1   class         3846 non-null   object
 2   conversation  3846 non-null   object
dtypes: int64(1), object(2)
memory usage: 120.2+ KB


In [11]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,973,973
기타 괴롭힘 대화,1011,1011
직장 내 괴롭힘 대화,970,970
협박 대화,892,892


In [12]:
train_data.iloc[29,2]

'병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 되지 너 공부 개못하는 찐따잖아.\n야 닭대가리가 말도 하네 동물농장 나가자.\n너네 학교폭력으로 신고하기 전에 그만 해.\n아이고 무서워라 살려주세요 으헝헝\n제발 좀 저리 가.\n나였으면 아이스크림 먹을 시간에 공부한다.\n멈춰!\n저러니까 쳐맞고 다니지'

In [13]:
import re

def preprocess_document(document):
    document = re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9]', ' ', document)
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


preprocess_document(train_data.iloc[29,2])

'병신이 아이스크림 먹게 돼 있냐 난 먹으면 안 돼 그만 좀 해 당연히 안 되지 너 공부 개못하는 찐따잖아 야 닭대가리가 말도 하네 동물농장 나가자 너네 학교폭력으로 신고하기 전에 그만 해 아이고 무서워라 살려주세요 으헝헝 제발 좀 저리 가 나였으면 아이스크림 먹을 시간에 공부한다 멈춰 저러니까 쳐맞고 다니지'

In [14]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

corpus = train_data['conversation'].map(preprocess_document)

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

tokenizer = LTokenizer(scores=scores)
tokenizer.tokenize(preprocess_document(train_data.iloc[29,2]))

training was done. used memory 0.242 Gbry 0.172 Gb
all cohesion probabilities was computed. # words = 13946
all branching entropies was computed # words = 19894
all accessor variety was computed # words = 19894


['병신',
 '이',
 '아이스크림',
 '먹게',
 '돼',
 '있냐',
 '난',
 '먹으면',
 '안',
 '돼',
 '그만',
 '좀',
 '해',
 '당연히',
 '안',
 '되지',
 '너',
 '공부',
 '개못하는',
 '찐따',
 '잖아',
 '야',
 '닭대가리가',
 '말도',
 '하네',
 '동물농장',
 '나가',
 '자',
 '너네',
 '학교',
 '폭력으로',
 '신고',
 '하기',
 '전에',
 '그만',
 '해',
 '아이고',
 '무서워',
 '라',
 '살려주세요',
 '으헝헝',
 '제발',
 '좀',
 '저리',
 '가',
 '나였으면',
 '아이스크림',
 '먹을',
 '시간',
 '에',
 '공부',
 '한다',
 '멈춰',
 '저러니',
 '까',
 '쳐맞고',
 '다니지']

In [15]:
class_names = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

train_data = train_data[['class', 'conversation']]
train_data['class'] = train_data['class'].map(lambda c: class_names.index(c))
train_data = train_data.reset_index()

train_data.head()

,index,class,conversation
0,0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


## 벡터화

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vocab_size = None

vector = TfidfVectorizer(preprocessor=preprocess_document, 
                         tokenizer=tokenizer.tokenize,
                         max_features=vocab_size)
print(vector.fit_transform(train_data['conversation']).toarray())
print("단어 개수:", len(vector.vocabulary_))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
단어 개수: 33715


In [17]:
x_data = vector.transform(train_data['conversation'])
y_data = train_data['class']

# train step 및 평가 함수

In [18]:
from sklearn.metrics import classification_report

def save_and_show_report(y_true, y_pred, prefix, show=True, zero_division=0):
    report = classification_report(y_true, y_pred, zero_division=zero_division)
    with open(BASE_DIR + '/' + prefix + '_report.txt', 'w') as f:
        f.write(report)
    if show:
        print(report)

In [19]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def graph_confusion_matrix(model, x_test, y_test, classes_name, prefix, show=True):
    df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)), index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12,12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
    plt.ylabel('label')
    plt.xlabel('predicted value')
    plt.savefig(BASE_DIR + '/' + prefix + '_confusion.png')
    if show:
        plt.show()
    else:
        plt.close()

In [20]:
def train_and_eval(model, x_train, y_train, x_val, y_val, prefix):
    model.fit(x_train, y_train)
    
    save_and_show_report(y_train, model.predict(x_train), prefix + '_train', show=False)
    graph_confusion_matrix(model, x_train, y_train, class_names, prefix + '_train', show=False)
    
    save_and_show_report(y_val, model.predict(x_val), prefix + '_val', show=False)
    graph_confusion_matrix(model, x_val, y_val, class_names, prefix + '_val', show=False)
    
    return model

In [21]:
def create_submission(model, prefix):
    submission_data = []

    for file_name, row in test_data.iterrows():
        text = row['text']
        _input = vector.transform([text])
        predicted = model.predict(_input)[0]

        submission_data.append({
            'file_name': file_name,
            'class': predicted
        })
    
    submission_data = pd.DataFrame(submission_data)
    submission_data.to_csv(prefix + '_submission.csv', index=False)

# 모델 학습 및 평가

In [22]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

models = [
    (MultinomialNB, dict()),
    (ComplementNB, dict()),
    (LinearSVC, {'C': 1000, 'penalty': 'l1', 'max_iter': 50000, 'dual': False}),
    (DecisionTreeClassifier, {'max_depth': 10}),
    (RandomForestClassifier, {'n_estimators': 10, 'max_depth': 10}),
    (GradientBoostingClassifier, dict())
]

In [21]:
from sklearn.model_selection import KFold

kfold = KFold(shuffle=True)

In [22]:
import os
import time


#BASE_DIR = time.strftime("%m%d-%H%M%S", time.localtime())
BASE_DIR = 'soynlp'
os.mkdir(BASE_DIR)

for Model, options in models:
    for i, (train_indice, val_indice) in enumerate(kfold.split(x_data)):
        model = Model(**options)
        
        x_train = x_data[train_indice]
        y_train = y_data[train_indice]
        x_val = x_data[val_indice]
        y_val = y_data[val_indice]
        
        prefix = Model.__name__ + '_' + str(i)
        model = train_and_eval(model, x_train, y_train, x_val, y_val, prefix)
        
        print(f'{prefix}: {accuracy_score(y_train, model.predict(x_train)):.4f}, {accuracy_score(y_val, model.predict(x_val)):.4f}')

MultinomialNB_0: 0.9613, 0.8571
MultinomialNB_1: 0.9630, 0.8622
MultinomialNB_2: 0.9591, 0.8531
MultinomialNB_3: 0.9617, 0.8427
MultinomialNB_4: 0.9626, 0.8505
ComplementNB_0: 0.9746, 0.8338
ComplementNB_1: 0.9737, 0.8674
ComplementNB_2: 0.9714, 0.8752
ComplementNB_3: 0.9727, 0.8752
ComplementNB_4: 0.9724, 0.8570
LinearSVC_0: 1.0000, 0.8506
LinearSVC_1: 1.0000, 0.8192
LinearSVC_2: 1.0000, 0.8166
LinearSVC_3: 1.0000, 0.8218
LinearSVC_4: 1.0000, 0.8036
DecisionTreeClassifier_0: 0.6925, 0.6390
DecisionTreeClassifier_1: 0.7049, 0.6112
DecisionTreeClassifier_2: 0.7199, 0.6151
DecisionTreeClassifier_3: 0.7095, 0.5839
DecisionTreeClassifier_4: 0.6948, 0.5631
RandomForestClassifier_0: 0.7207, 0.6325
RandomForestClassifier_1: 0.7309, 0.6138
RandomForestClassifier_2: 0.7150, 0.5995
RandomForestClassifier_3: 0.7436, 0.6177
RandomForestClassifier_4: 0.7332, 0.6177
GradientBoostingClassifier_0: 0.9330, 0.7792
GradientBoostingClassifier_1: 0.9308, 0.8336
GradientBoostingClassifier_2: 0.9340, 0.7789


In [24]:
BASE_DIR = 'soynlp'

options = dict()
#options = {'C': 1000, 'penalty': 'l1', 'max_iter': 50000, 'dual': False}
model = ComplementNB(**options)
model = train_and_eval(model, x_data, y_data, x_data, y_data, 'CNB_final')
create_submission(model, 'soynlp_CNB')

In [24]:
word_score_table.keys()

dict_keys(['닌', '껄', '나', '뱃', '납', '값', '독', '회', '희', '학', '총', '고', '능', '얌', '뽑', '온', '란', '측', '권', '큰', '봉', '건', '본', '질', '네', '갈', '합', '냅', '몰', '뵈', '한', '닿', '벽', '났', '월', '전', '혼', '쳤', '깎', '닮', '뻔', '증', '래', '포', '꿔', '뻗', '힌', '쎄', '넉', '격', '녹', '우', '롱', '써', '줄', '답', '폰', '2', '맨', '량', '빡', '소', '육', '승', '형', '법', '꼰', '쿵', '토', '주', '설', '씹', '엿', '쯧', '블', '틴', '붙', '슈', '딸', '흐', '멈', '뿌', '손', '팀', '쩜', '추', '착', '현', '흥', '띠', '별', '잃', '름', '응', '놈', '있', '쓰', '극', '꿇', '말', '갇', '죽', '튀', '셔', '해', '끓', '졌', '꺄', '왠', '솔', '서', '히', '갖', '외', '옷', '두', '김', '땀', '웃', '썼', '씩', '웩', '석', '즉', '변', '럴', '핳', '껴', '사', '띄', '녁', '찢', '떡', '낮', '냄', '영', '녀', '컸', '워', '득', '린', '인', '깝', '쪼', '엄', '플', '톱', '싶', '층', '남', '퉁', '했', '징', '쩔', '시', '꽉', '자', '옥', '쳐', '빌', '밑', '궁', '긁', '교', '심', '텅', '뭔', '켜', '참', '데', '쭈', '탈', '뻥', '예', '습', '못', '북', '세', '챙', '멍', '폐', '언', '썻', '핫', '넥', '활', '어', '섭', '루', '연', '림', '찰', '차', '잇', '믿', '가', '여', '둬',

In [25]:
test_corpus = test_data['text'].map(preprocess_document)

In [26]:
test_seqs = test_corpus.map(tokenizer.tokenize)
test_seqs

t_000    [아가씨, 담배, 한갑주소, 네, 450, 0원입니다, 어, 네, 지갑어디갔지, 에...
t_001    [우리, 팀에서, 다른, 팀으로, 갈, 사람, 없나, 그럼, 영지씨가, 가는건, 어...
t_002    [너, 오늘, 그게, 뭐야, 네, 제가, 뭘, 잘못했, 나요, 제대로, 좀, 하지,...
t_004    [아무튼, 앞으로, 니가, 내, 와이, 파이야, 응, 와이, 파이, 온, 켰어, 반...
t_005    [그러니까, 빨리, 말해, 선생님, 제발, 살려주, 십시오, 비밀번호, 틀릴, 때마...
                               ...                        
t_495    [미나씨, 휴가, 결제, 올리, 기, 전에, 저랑, 상의하라고, 말한거, 기억해, ...
t_496    [교수님, 제, 논문, 에, 제, 이름이, 없나요, 아, 무슨, 논문, 말이야, 지...
t_497    [야, 너, 네, 저요, 그래, 너, 왜요, 돈좀, 줘봐, 돈, 없어, 요, 돈이,...
t_498    [야, 너, 빨리, 안, 뛰어, 와, 너, 이, 환자, 제대로, 봤어, 안, 봤어,...
t_499    [엄마, 저, 그, 돈, 안해주, 시면, 정말, 큰일, 나요, 이유도, 말하지, 않...
Name: text, Length: 400, dtype: object

In [27]:
vocabulary = word_score_table.keys()

total_words = 0
coverage = 0

for row in test_seqs:
    total_words += len(row)
    for word in row:
        if word in vocabulary:
            coverage += 1
            
print('test coverage:', coverage / float(total_words))

test coverage: 0.7697015245866438


In [2]:
results = dict()

with open('SoyNLP.txt', 'r') as f:
    while True:
        line = f.readline()
        if line is None or line == '':
            break
        model_name, train_acc, val_acc = line.split()
        model_name = model_name[:-3]
        train_acc = float(train_acc[:-1])
        val_acc = float(val_acc[:-1])
        if results.get(model_name, None) is None:
            results[model_name] = {'train': 0.0, 'val': 0.0}
        results[model_name]['train'] += train_acc
        results[model_name]['val'] += val_acc

for model_name in results:
    print(model_name + ":")
    print(f"\ttrain: {results[model_name]['train'] / 5.0:.4f}")
    print(f"\tval: {results[model_name]['val'] / 5.0:.4f}")

MultinomialNB:
	train: 0.9615
	val: 0.8528
ComplementNB:
	train: 0.9730
	val: 0.8614
LinearSVC:
	train: 1.0000
	val: 0.8218
DecisionTreeClassifier:
	train: 0.7043
	val: 0.6022
RandomForestClassifier:
	train: 0.7287
	val: 0.6156
GradientBoostingClassifier:
	train: 0.9320
	val: 0.7926
